### Создание таблиц

In [1]:
import psycopg2
import os
from dotenv import load_dotenv

# Загрузка переменных окружения из .env файла
load_dotenv()

# Получение значений переменных окружения
db_name = os.getenv('POSTGRES_DB_FAKE')
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

# Подключение к базе данных PostgreSQL
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,
)
cursor = conn.cursor()

# SQL-запрос для создания таблицы employe
create_employe_table_query = """
CREATE TABLE IF NOT EXISTS employe (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    middle_name VARCHAR(50),
    gender VARCHAR(10),
    country VARCHAR(50),
    address TEXT,
    phone VARCHAR(20),
    email VARCHAR(100),
    birth_date DATE,
    bank VARCHAR(100),
    company VARCHAR(100),
    username VARCHAR(50),
    inn VARCHAR(20)
);
"""

# SQL-запрос для создания таблицы transactions
create_transactions_table_query = """
CREATE TABLE IF NOT EXISTS transactions (
    id SERIAL PRIMARY KEY,
    user_id INTEGER REFERENCES employe(id),
    timestamp TIMESTAMP,
    stock_name VARCHAR(100),
    price NUMERIC
);
"""

# SQL-запрос для создания таблицы transaction_inn
create_transaction_inn_table_query = """
CREATE TABLE IF NOT EXISTS transaction_inn (
    id SERIAL PRIMARY KEY,
    inn_from VARCHAR(20),
    inn_to VARCHAR(20),
    amount NUMERIC
);
"""

# Выполнение запросов
cursor.execute(create_employe_table_query)
cursor.execute(create_transactions_table_query)
cursor.execute(create_transaction_inn_table_query)

# Подтверждение изменений
conn.commit()

# Закрытие курсора и соединения
cursor.close()
conn.close()

print("Tables created successfully")


Tables created successfully


#### Заполнение таблиц

In [38]:
import os
import random
from datetime import datetime

import psycopg2
from dotenv import load_dotenv
from mimesis import Generic
from mimesis.enums import Gender
from mimesis.locales import Locale
from mimesis.builtins import RussiaSpecProvider
from sqlalchemy import create_engine, Column, Integer, String, Date, Numeric, ForeignKey, Text, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from tqdm import tqdm

# Load environment variables from .env file
load_dotenv()

# Get environment variables
db_name = os.getenv('POSTGRES_DB_FAKE')
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

# Define the database URL
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create SQLAlchemy engine and session
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# Define the base class for declarative models
Base = declarative_base()

# Define the Employee model
class Employee(Base):
    __tablename__ = 'employe'
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    middle_name = Column(String(50))
    gender = Column(String(10))
    country = Column(String(50))
    address = Column(Text)
    phone = Column(String(20))
    email = Column(String(100))
    birth_date = Column(Date)
    bank = Column(String(100))
    company = Column(String(100))
    username = Column(String(50))
    inn = Column(String(20))

# Define the Transaction model
class Transaction(Base):
    __tablename__ = 'transactions'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('employe.id'))
    timestamp = Column(DateTime)
    stock_name = Column(String(100))
    price = Column(Numeric)

# Define the TransactionInn model
class TransactionInn(Base):
    __tablename__ = 'transaction_inn'
    id = Column(Integer, primary_key=True)
    inn_from = Column(String(20))
    inn_to = Column(String(20))
    amount = Column(Numeric)

# Create tables in the database
Base.metadata.create_all(engine)

class FakeDataGenerator:
    def __init__(self, num_employees, num_transactions, num_transaction_inns, batch_size=1000):
        self.num_employees = num_employees
        self.num_transactions = num_transactions
        self.num_transaction_inns = num_transaction_inns
        self.batch_size = batch_size
        self.generic = Generic(locale=Locale.RU)
        self.russia_provider = RussiaSpecProvider()
        self.session = session

    def generate_employee_data(self):
        employees = []
        for _ in tqdm(range(self.num_employees), desc="Generating employee data"):
            gender = self.generic.choice([Gender.MALE, Gender.FEMALE])
            employees.append(Employee(
                first_name=self.generic.person.first_name(gender),
                last_name=self.generic.person.last_name(gender),
                middle_name=self.generic.person.full_name(gender),
                gender=gender.value,
                country=self.generic.address.country(),
                address=self.generic.address.address(),
                phone=self.generic.person.phone_number(),
                email=self.generic.person.email(),
                birth_date=self.generic.datetime.date(),
                bank=self.generic.finance.bank(),
                company=self.generic.finance.company(),
                username=self.generic.person.username(),
                inn=self.russia_provider.inn()
            ))
        return employees

    def generate_transaction_data(self, employee_ids):
        transactions = []
        for _ in tqdm(range(self.num_transactions), desc="Generating transaction data"):
            transactions.append(Transaction(
                user_id=random.choice(employee_ids),
                timestamp=self.generic.datetime.datetime(),
                stock_name=self.generic.finance.stock_name(),
                price=self.generic.finance.price()
            ))
        return transactions

    def generate_transaction_inn_data(self, inn_list):
        transaction_inns = []
        num_inns = len(inn_list)
        inn_from_sample = random.sample(inn_list, int(num_inns * 0.3))
        inn_to_sample = random.sample(inn_list, int(num_inns * 0.3))
        for _ in tqdm(range(self.num_transaction_inns), desc="Generating transaction_inn data"):
            transaction_inns.append(TransactionInn(
                inn_from=random.choice(inn_from_sample),
                inn_to=random.choice(inn_to_sample),
                amount=self.generic.finance.price()
            ))
        return transaction_inns

    def insert_employee_data(self, employees):
        for i in tqdm(range(0, len(employees), self.batch_size), desc="Inserting employee data"):
            batch = employees[i:i + self.batch_size]
            self.session.bulk_save_objects(batch)
            self.session.commit()
        return [employee.id for employee in employees]

    def insert_transaction_data(self, transactions):
        for i in tqdm(range(0, len(transactions), self.batch_size), desc="Inserting transaction data"):
            batch = transactions[i:i + self.batch_size]
            self.session.bulk_save_objects(batch)
            self.session.commit()

    def insert_transaction_inn_data(self, transaction_inns):
        for i in tqdm(range(0, len(transaction_inns), self.batch_size), desc="Inserting transaction_inn data"):
            batch = transaction_inns[i:i + self.batch_size]
            self.session.bulk_save_objects(batch)
            self.session.commit()

    def generate_and_insert_data(self):
        employees = self.generate_employee_data()
        employee_ids = self.insert_employee_data(employees)
        inn_list = [employee.inn for employee in employees]
        transactions = self.generate_transaction_data(employee_ids)
        self.insert_transaction_data(transactions)
        transaction_inns = self.generate_transaction_inn_data(inn_list)
        self.insert_transaction_inn_data(transaction_inns)

    def close_connection(self):
        self.session.close()

if __name__ == '__main__':
    num_employees = 10_000_000  # specify the number of employee records to generate
    num_transactions = 10  # specify the number of transaction records to generate
    num_transaction_inns = 10  # specify the number of transaction_inn records to generate
    generator = FakeDataGenerator(num_employees, num_transactions, num_transaction_inns, batch_size=10_000)
    generator.generate_and_insert_data()
    generator.close_connection()
    print("Data generated and inserted successfully.")


/var/folders/2v/xnsq11qs6gz1r5nfrmhckglw0000gn/T/ipykernel_92944/1922801101.py:35: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
Inserting transaction_inn data: 100%|██████████| 1/1 [00:00<00:00, 134.85it/s]


Data generated and inserted successfully.


In [39]:
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.types import Integer, String, Date, Numeric, Text
from dotenv import load_dotenv
from mimesis import Generic
from mimesis.enums import Gender
from mimesis.locales import Locale
from mimesis.builtins import RussiaSpecProvider
from tqdm import tqdm
import logging
import time

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Загрузка переменных окружения из .env файла
load_dotenv()

# Получение значений переменных окружения
db_name = os.getenv('POSTGRES_DB_FAKE')
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

# Создание подключения к базе данных с использованием SQLAlchemy
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
metadata = MetaData()

# Инициализация генератора данных
generic = Generic(locale=Locale.RU)
russia_provider = RussiaSpecProvider()

# Определение колонок таблицы
columns = ['id', 'first_name', 'last_name', 'middle_name', 'gender', 'country', 'address', 'phone', 'email', 'birth_date', 'bank', 'company', 'username', 'inn']

# Типы данных для столбцов
dtype_map = {
    'id': Integer,
    'first_name': String(50),
    'last_name': String(50),
    'middle_name': String(50),
    'gender': String(10),
    'country': String(50),
    'address': Text,
    'phone': String(20),
    'email': String(100),
    'birth_date': Date,
    'bank': String(100),
    'company': String(100),
    'username': String(50),
    'inn': String(20)
}

# Функция для генерации нового значения на основе типа данных
def generate_new_value(column, old_value):
    if column == 'first_name':
        gender = Gender.MALE if generic.choice([True, False]) else Gender.FEMALE
        return generic.person.first_name(gender)
    elif column == 'last_name':
        return generic.person.last_name()
    elif column == 'middle_name':
        gender = Gender.MALE if generic.choice([True, False]) else Gender.FEMALE
        return generic.person.full_name(gender)
    elif column == 'gender':
        return generic.choice(['Male', 'Female'])
    elif column == 'country':
        return generic.address.country()
    elif column == 'address':
        return generic.address.address()
    elif column == 'phone':
        return generic.person.phone_number()
    elif column == 'email':
        return generic.person.email()
    elif column == 'birth_date':
        return generic.datetime.date()
    elif column == 'bank':
        return generic.finance.bank()
    elif column == 'company':
        return generic.finance.company()
    elif column == 'username':
        return generic.person.username()
    elif column == 'inn':
        return russia_provider.inn()
    else:
        return old_value

# Создание сопоставления старых и новых значений
def create_mapping(df):
    mapping = {}
    for column in tqdm(columns, desc="Creating mapping"):
        if column != 'id':
            unique_values = df[column].unique()
            mapping[column] = {value: generate_new_value(column, value) for value in unique_values}
    return mapping

# Применение сопоставления для создания новой таблицы с деперсонализированными данными
def apply_mapping(df, mapping):
    for column in columns:
        if column != 'id':
            df[column] = df[column].map(mapping[column])
    return df

# Основная функция
def main():
    # Загрузка данных из таблицы employe в DataFrame
    start_time = time.time()
    logger.info("Loading data...")
    employe_table = Table('employe', metadata, autoload_with=engine)
    df = pd.read_sql(employe_table.select(), con=engine)
    logger.info(f"Data loaded in {time.time() - start_time:.2f} seconds")

    # Создание сопоставления и применение его к данным
    start_time = time.time()
    mapping = create_mapping(df)
    logger.info(f"Mapping created in {time.time() - start_time:.2f} seconds")

    start_time = time.time()
    df_fake = apply_mapping(df, mapping)
    logger.info(f"Mapping applied in {time.time() - start_time:.2f} seconds")

    # Запись деперсонализированных данных в новую таблицу
    start_time = time.time()
    df_fake.to_sql('employe_fake', con=engine, if_exists='replace', index=False, dtype=dtype_map)
    logger.info(f"Data written to employe_fake in {time.time() - start_time:.2f} seconds")

if __name__ == '__main__':
    main()
    session.close()
    logger.info("Data depersonalized successfully.")


INFO:__main__:Loading data...
INFO:__main__:Data loaded in 5.91 seconds
Creating mapping: 100%|██████████| 14/14 [00:35<00:00,  2.53s/it]
INFO:__main__:Mapping created in 35.47 seconds
INFO:__main__:Mapping applied in 5.36 seconds
INFO:__main__:Data written to employe_fake in 69.23 seconds
INFO:__main__:Data depersonalized successfully.
